In [21]:
%run -i ../header.py
from pylatex import Tabular, MultiColumn, NoEscape, LongTable

In [22]:
data_pack = {
      "b": ["b-trans.deng2017-"+num for num in ("III", "IV", "V", "VI")]
    , "c": ["c-trans.deng2016-"+num for num in ("V", "VI", "VII", "VIII", "IX")]
}

# Export

In [23]:
def getPDGTrans(name):
    # return widths in [KeV]
    pdg_trans = pd.read_csv("../data/{}-trans.PDG.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]\
                  .rename("PDG")
    pdg = pd.read_csv("../data/{}-widths.csv".format(name), header=0, index_col=0, comment="#")["PDG"]
    pdg_spec = readSpec("../data/{}-widths.csv".format(name))
    pdg_err = pd.read_csv("../data/{}-trans.PDG.err.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]\
                  .rename("PDG_err")
    for instate in pdg_trans.index.levels[0]:
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg[instate]*dimTrans(pdg_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
        pdg_err.loc[(instate, slice(None))] = pdg_err[(instate, slice(None))].values*pdg[instate]*dimTrans(pdg_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    pdg_err.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_err.index.values], name=["In", "Out"])
    pdg_err = pdg_err.reindex(pdg_trans.index)
    return pdg_trans, pdg_err.abs()

In [24]:
def getDengTrans(prefix, cfgname):
    # return trans widths of Deng in KeV
    prefixToCol = {"scr": "SP", "lin": "LP"}
    
    col = prefixToCol[prefix]
    spec = readSpec("../data/{}.csv".format(cfgname))
    deng = pd.read_csv("../data/{}.csv".format(cfgname), header=0, index_col=[0,1], comment="#")
    for k, v in spec.items():
        deng[k] *= dimTrans(v["tags"][0], "KeV")
    deng.rename(columns={col: "Deng"}, inplace=True)
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

In [25]:
def getRs(particle, prefix, cfgname, suffix=""):
    # return my widths in KeV
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1], comment="#")
    data *= dimTrans("GeV", "KeV")
    data = data.loc[(data != 0).any(axis=1), :]
    data["in0"] /= data["width"]
    data["in2"] /= data["width"]
    data["out0"] /= data["width"]
    data["out2"] /= data["width"]
    data.index.names = ["In", "Out"]
    
    data_sub = data.query("In == '{0}'".format(particle))\
        .rename(columns={"out0": "$r^{(0)}$", "out2": "$r^{(2)}$"})\
        .drop(columns=["in0", "in2"])
    data_above = data.query("Out == '{0}'".format(particle))\
        .rename(columns={"in0": "$r^{(0)}$", "in2": "$r^{(2)}$"})\
        .drop(columns=["out0", "out2"])
    
    data = pd.concat([data_sub, data_above])
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data

In [27]:
def reportRs(pname, prefix, cfgname, suffixes=["", "E1"]): # allowed suffixes: "", "E1", "ELW"
    stack = []
    deng_stack = []
    parts = [getRs(pname, prefix, cfgname, suffix)\
                                .add_prefix("" if suffix == "" else suffix+"-")\
                            for suffix in suffixes]
    parts = [part for part in parts if not part.empty]
    if len(parts) > 0:
        stack.append(pd.concat(parts, axis=1))
    
    for dataf in data_pack[cfgname.split("-")[0]]:
        deng_stack.append(getDengTrans(prefix, dataf)["Deng"])
    
    stack = pd.concat(stack, sort=False)
    deng_stack = pd.concat(deng_stack, axis=0, sort=False).reindex(stack.index.values)
    pdg, _ = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(stack.index.values)
    
    res = pd.concat([pdg, deng_stack, stack], sort=False, axis=1)
    res = dfsort(res, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                      ,[name2code(i[1:-1]) for i in a]
                                                      ,[name2code(i[1:-1]) for i in b]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [28]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,2.78,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,147,179,144,1,0,179,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,331,0.571,0,378,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,367,292,501,0.142,0.519,434,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,1.64,1,0,2.68,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,88.8,0.629,0,112,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,247,0.17,0.47,194,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,147,179,179,1,0,187,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,378,0.5,0,411,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,367,292,434,0.1,0.6,481,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,2.68,1,0,3.15,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,112,0.5,0,60.1,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,194,0.1,0.6,136,0.1,0.6


In [29]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,2.57,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,147,172,140,1,0,172,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,311,0.566,0,352,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,367,284,477,0.14,0.523,416,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,3.03,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,2.89,1,0,4.63,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,90.9,0.625,0,114,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,270,0.169,0.473,213,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,147,172,172,1,0,180,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,352,0.5,0,380,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,367,284,416,0.1,0.6,458,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,4.63,1,0,1.71,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,114,0.5,0,58,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,213,0.1,0.6,146,0.1,0.6


In [30]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_1S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_1S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0166,1,0,0.0166,1,0
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,25.2,1,0,27.4,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,32.4,0.523,0,33.9,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,38.4,0.114,0.573,36.6,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.23,1,0,4.94,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,10.5,0.54,0,11.4,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,16.7,0.123,0.554,15.4,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.38,1,0,1.69,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,5.63,0.553,0,6.22,0.5,0


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0166,1,0,nan,nan,nan
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,27.4,1,0,28,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,33.9,0.5,0,34.9,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,36.6,0.1,0.6,37.9,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.94,1,0,3.8,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,11.4,0.5,0,9.82,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,15.4,0.1,0.6,13.8,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.69,1,0,1.02,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,6.22,0.5,0,4.88,0.5,0


### $\psi(2S)$ and $\Upsilon(2S)$

In [31]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8      10.3          1          0   
                $\chi_{c0}(1P)$    29    22      33.1          1          0   
                $\chi_{c1}(1P)$  28.9    45      40.9      0.474          0   
                $\chi_{c2}(1P)$  28.2    46      40.6     0.0872      0.626   
                $\eta_{c}(2S)$  0.207  0.19     0.148          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99      99.1          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155       193      0.541          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150       242      0.126       0.55   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1      7.25          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74      71.5      0.586          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76       142      0.149      0.507   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          10.3             1             0  
                $\chi_{c0}(1P)$         28.1             1             0  
                $\chi_{c1}(1P)$         38.8           0.5             0  
                $\chi_{c2}(1P)$         42.4           0.1           0.6  
                $\eta_{c}(2S)$         0.148             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               112             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               208           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               222           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              9.98             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$              83.8           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               120           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8         10.3             1   
                $\chi_{c0}(1P)$    29    22         28.1             1   
                $\chi_{c1}(1P)$  28.9    45         38.8           0.5   
                $\chi_{c2}(1P)$  28.2    46         42.4           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.148             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99          112             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155          208           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150          222           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1         9.98             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74         83.8           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76          120           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          30.3              1   
                $\chi_{c1}(1P)$             0          39.9            0.5   
                $\chi_{c2}(1P)$           0.6          43.2            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           117              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           231            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           254            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0        0.0413              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          41.7            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6            78            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [32]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08      9.63          1          0   
                $\chi_{c0}(1P)$    29    22      33.5          1          0   
                $\chi_{c1}(1P)$  28.9    42      41.9      0.474          0   
                $\chi_{c2}(1P)$  28.2    38      38.9     0.0877      0.625   
                $\eta_{c}(2S)$  0.207  0.19     0.158          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121       120          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139       223      0.542          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135       288      0.127      0.548   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17      11.3          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94      90.7      0.596          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97       201      0.155      0.497   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          9.63             1             0  
                $\chi_{c0}(1P)$         28.6             1             0  
                $\chi_{c1}(1P)$         39.8           0.5             0  
                $\chi_{c2}(1P)$         40.5           0.1           0.6  
                $\eta_{c}(2S)$         0.158             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               136             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               242           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               263           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              16.2             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$               108           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               166           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08         9.63             1   
                $\chi_{c0}(1P)$    29    22         28.6             1   
                $\chi_{c1}(1P)$  28.9    42         39.8           0.5   
                $\chi_{c2}(1P)$  28.2    38         40.5           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.158             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121          136             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139          242           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135          263           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17         16.2             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94          108           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97          166           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          30.7              1   
                $\chi_{c1}(1P)$             0          40.9            0.5   
                $\chi_{c2}(1P)$           0.6          41.2            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           143              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           269            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           303            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0         0.484              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          38.8            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          89.6            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [33]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_2S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_2S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066    0.0745          1          0   
                $\chi_{b0}(1P)$   1.22  1.09      0.92          1          0   
                $\chi_{b1}(1P)$   2.21  2.17      1.45      0.494          0   
                $\chi_{b2}(1P)$   2.29  2.62      1.56     0.0971      0.606   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4      12.5          1          0   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3      15.7      0.512          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3      17.4      0.108      0.585   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55      2.31          1          0   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63      5.44      0.526          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72      7.89      0.115       0.57   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$        0.0745             1             0  
                $\chi_{b0}(1P)$        0.892             1             0  
                $\chi_{b1}(1P)$         1.43           0.5             0  
                $\chi_{b2}(1P)$         1.58           0.1           0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$          13.1             1             0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$          16.1           0.5             0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$          16.9           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$          2.54             1             0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$          5.72           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$           7.5           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066       0.0745             1   
                $\chi_{b0}(1P)$   1.22  1.09        0.892             1   
                $\chi_{b1}(1P)$   2.21  2.17         1.43           0.5   
                $\chi_{b2}(1P)$   2.29  2.62         1.58           0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4         13.1             1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3         16.1           0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3         16.9           0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55         2.54             1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63         5.72           0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72          7.5           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0         0.899              1   
                $\chi_{b1}(1P)$             0          1.44            0.5   
                $\chi_{b2}(1P)$           0.6          1.58            0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$              0          13.4              1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$              0          16.6            0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$            0.6          17.6            0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$              0           1.7              1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$              0          4.53            0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$            0.6          6.26            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(2S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$             0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$             0.6

### $\psi(3S)$ and $\Upsilon(3S)$

In [34]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76      9.62          1          0   
           $\chi_{c0}(1P)$  nan   6.7      19.2          1          0   
           $\chi_{c1}(1P)$  nan   6.7      9.78      0.427          0   
           $\chi_{c2}(1P)$  nan   2.5      1.93     0.0568      0.693   
           $\eta_{c}(2S)$   nan  1.78      2.01          1          0   
           $\chi_{c0}(2P)$  nan    27      32.1          1          0   
           $\chi_{c1}(2P)$  nan    67      45.7       0.48          0   
           $\chi_{c2}(2P)$  nan    82      45.2     0.0904      0.619   
           $\eta_{c}(3S)$   nan 0.088    0.0344          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.62             1             0  
           $\chi_{c0}(1P)$         13.7             1             0  
           $\chi_{c1}(1P)$         8.35           0.5             0  
           $\chi_{c2}(1P)$         2.23           0.1           0.6  
           $\eta_{c}(2S)$          2.01             1             0  
           $\chi_{c0}(2P)$         28.5             1             0  
           $\chi_{c1}(2P)$         43.9           0.5             0  
           $\chi_{c2}(2P)$         46.7           0.1           0.6  
           $\eta_{c}(3S)$        0.0344             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76         9.62             1   
           $\chi_{c0}(1P)$  nan   6.7         13.7             1   
           $\chi_{c1}(1P)$  nan   6.7         8.35           0.5   
           $\chi_{c2}(1P)$  nan   2.5         2.23           0.1   
           $\eta_{c}(2S)$   nan  1.78         2.01             1   
           $\chi_{c0}(2P)$  nan    27         28.5             1   
           $\chi_{c1}(2P)$  nan    67         43.9           0.5   
           $\chi_{c2}(2P)$  nan    82         46.7           0.1   
           $\eta_{c}(3S)$   nan 0.088       0.0344             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          7.73              1   
           $\chi_{c1}(1P)$             0         0.675            0.5   
           $\chi_{c2}(1P)$           0.6          1.92            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          31.4              1   
           $\chi_{c1}(2P)$             0          45.7            0.5   
           $\chi_{c2}(2P)$           0.6          47.9            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [35]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66      10.4          1          0   
           $\chi_{c0}(1P)$  nan   5.9      21.6          1          0   
           $\chi_{c1}(1P)$  nan     4      10.1      0.425          0   
           $\chi_{c2}(1P)$  nan  0.25     0.775     0.0552      0.696   
           $\eta_{c}(2S)$   nan  1.65       2.8          1          0   
           $\chi_{c0}(2P)$  nan    19      42.9          1          0   
           $\chi_{c1}(2P)$  nan    55      65.3      0.478          0   
           $\chi_{c2}(2P)$  nan    67      63.4     0.0891      0.622   
           $\eta_{c}(3S)$   nan 0.051    0.0527          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          10.4             1             0  
           $\chi_{c0}(1P)$         15.3             1             0  
           $\chi_{c1}(1P)$         8.54           0.5             0  
           $\chi_{c2}(1P)$          0.9           0.1           0.6  
           $\eta_{c}(2S)$           2.8             1             0  
           $\chi_{c0}(2P)$         37.8             1             0  
           $\chi_{c1}(2P)$         62.4           0.5             0  
           $\chi_{c2}(2P)$         65.7           0.1           0.6  
           $\eta_{c}(3S)$        0.0527             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66         10.4             1   
           $\chi_{c0}(1P)$  nan   5.9         15.3             1   
           $\chi_{c1}(1P)$  nan     4         8.54           0.5   
           $\chi_{c2}(1P)$  nan  0.25          0.9           0.1   
           $\eta_{c}(2S)$   nan  1.65          2.8             1   
           $\chi_{c0}(2P)$  nan    19         37.8             1   
           $\chi_{c1}(2P)$  nan    55         62.4           0.5   
           $\chi_{c2}(2P)$  nan    67         65.7           0.1   
           $\eta_{c}(3S)$   nan 0.051       0.0527             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0           7.3              1   
           $\chi_{c1}(1P)$             0         0.105            0.5   
           $\chi_{c2}(1P)$           0.6           7.1            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          41.9              1   
           $\chi_{c1}(2P)$             0          65.4            0.5   
           $\chi_{c2}(2P)$           0.6          67.7            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [36]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_3S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_3S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071    0.0788          1          0   
                $\chi_{b0}(1P)$ 0.0549  0.097    0.0994          1          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005  8.09e-05      0.478          0   
                $\chi_{b2}(1P)$  0.201   0.14     0.149     0.0873      0.626   
                $\eta_{b}(2S)$     nan  0.011    0.0118          1          0   
                $\chi_{b0}(2P)$    1.2   1.21      1.31          1          0   
                $\chi_{b1}(2P)$   2.56   2.61       2.2      0.495          0   
                $\chi_{b2}(2P)$   2.66   3.16      2.44     0.0975      0.605   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      9.65          1          0   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      12.3      0.509          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      13.6      0.106      0.589   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0788             1             0  
                $\chi_{b0}(1P)$       0.0905             1             0  
                $\chi_{b1}(1P)$     7.73e-05           0.5             0  
                $\chi_{b2}(1P)$        0.156           0.1           0.6  
                $\eta_{b}(2S)$        0.0118             1             0  
                $\chi_{b0}(2P)$         1.28             1             0  
                $\chi_{b1}(2P)$         2.18           0.5             0  
                $\chi_{b2}(2P)$         2.47           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          9.96             1             0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          12.6           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          13.3           0.1           0.6

['E1', 'ELW']


PDG   Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071       0.0788             1   
                $\chi_{b0}(1P)$ 0.0549  0.097       0.0905             1   
                $\chi_{b1}(1P)$ 0.0183 0.0005     7.73e-05           0.5   
                $\chi_{b2}(1P)$  0.201   0.14        0.156           0.1   
                $\eta_{b}(2S)$     nan  0.011       0.0118             1   
                $\chi_{b0}(2P)$    1.2   1.21         1.28             1   
                $\chi_{b1}(2P)$   2.56   2.61         2.18           0.5   
                $\chi_{b2}(2P)$   2.66   3.16         2.47           0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95         9.96             1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3         12.6           0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8         13.3           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(3S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0        0.0392              1   
                $\chi_{b1}(1P)$             0        0.0357            0.5   
                $\chi_{b2}(1P)$           0.6         0.394            0.1   
                $\eta_{b}(2S)$              0           nan            nan   
                $\chi_{b0}(2P)$             0           1.3              1   
                $\chi_{b1}(2P)$             0           2.2            0.5   
                $\chi_{b2}(2P)$           0.6          2.48            0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$              0          10.2              1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$              0            13            0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$            0.6          13.9            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(3S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
                $\eta_{b}(2S)$             nan  
                $\chi_{b0}(2P)$              0  
                $\chi_{b1}(2P)$              0  
                $\chi_{b2}(2P)$            0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             0.6

### $\psi_1(1D)$ which is $\psi(3770)$

In [37]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.427          1          0   
                $\chi_{c0}(1P)$  188   261       275          1          0   
                $\chi_{c1}(1P)$ 67.7   135       119      0.539          0   
                $\chi_{c2}(1P)$  nan   8.1      5.74      0.253      0.531   
                $\eta_{c}(2S)$   nan   nan  0.000214          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12      7.22          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8      12.4      0.484          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46     0.747       0.12      0.694   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39     0.627          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2      3.63      0.443          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79      1.26      0.173     0.0518   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.427             1             0  
                $\chi_{c0}(1P)$          311             1             0  
                $\chi_{c1}(1P)$          143           0.5             0  
                $\chi_{c2}(1P)$         7.76           0.1           0.6  
                $\eta_{c}(2S)$      0.000214             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          7.08             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          11.4           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.607           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          0.56             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          2.84           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.218           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.427             1   
                $\chi_{c0}(1P)$  188   261          311             1   
                $\chi_{c1}(1P)$ 67.7   135          143           0.5   
                $\chi_{c2}(1P)$  nan   8.1         7.76           0.1   
                $\eta_{c}(2S)$   nan   nan     0.000214             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12         7.08             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8         11.4           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46        0.607           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39         0.56             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2         2.84           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79        0.218           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           349              1   
                $\chi_{c1}(1P)$             0           153            0.5   
                $\chi_{c2}(1P)$           0.6          8.22            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          7.12              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          11.8            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.638            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.48              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.355            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0719            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

In [38]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.365          1          0   
                $\chi_{c0}(1P)$  188   272       273          1          0   
                $\chi_{c1}(1P)$ 67.7   138       115      0.537          0   
                $\chi_{c2}(1P)$  nan   7.1      5.28      0.235      0.535   
                $\eta_{c}(2S)$   nan   nan  0.000138          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20      17.7          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9      18.3      0.482          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36      1.09      0.131      0.712   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27     0.126          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2      6.71      0.434          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5       2.7      0.201     0.0341   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.365             1             0  
                $\chi_{c0}(1P)$          307             1             0  
                $\chi_{c1}(1P)$          137           0.5             0  
                $\chi_{c2}(1P)$         7.03           0.1           0.6  
                $\eta_{c}(2S)$      0.000138             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          17.2             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          16.7           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.849           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          0.11             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          5.07           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.363           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.365             1   
                $\chi_{c0}(1P)$  188   272          307             1   
                $\chi_{c1}(1P)$ 67.7   138          137           0.5   
                $\chi_{c2}(1P)$  nan   7.1         7.03           0.1   
                $\eta_{c}(2S)$   nan   nan     0.000138             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20         17.2             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9         16.7           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36        0.849           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27         0.11             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2         5.07           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5        0.363           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           339              1   
                $\chi_{c1}(1P)$             0           146            0.5   
                $\chi_{c2}(1P)$           0.6          7.39            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          17.4              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          17.4            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.908            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.75              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.328            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0873            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

## Sumrules for the thesis

In [39]:
def getMasses(prefix, cfgname):
    # return masses in MeV
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    mass_data["E"] *= dimTrans("GeV", "MeV")
    pdg_spec = readSpec("../data/{}-spectrum.csv".format(cfgname.split("-")[0]))["PDG"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgname.split("-")[0]), header=0, index_col=0, comment="#")["PDG"]\
        *dimTrans(pdg_spec["tags"][0], "MeV")
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [40]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame(OrderedDict([(particle, values["eq"]) for particle, values in pprops.items()])).T
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    return props

In [41]:
def getSumrule(pname, prefix, cfgname, suffix=""):
    msuffix = "" if suffix == "" else suffix+"-" 
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgname, [suffix])
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**3
    sr = sr.reindex(widths.index.values)
    
    widths["$SR-PDG$"] = widths["PDG"].values
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$SR-PDG$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-PDG$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-Deng$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$SR-Deng$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-Deng$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-\Gamma$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    
    sigma = (widths["$SR-Deng$"] - widths["$SR-\Gamma$"]).std()
    
    return widths, pd.DataFrame(widths[["$SR-PDG$", "$SR-Deng$", "$SR-\Gamma$"]].sum().rename("$SR$")), sigma

In [42]:
def reportSumrules(pname, prefix, cfgname, suffixes=("", "E1", "ELW"), sigma=0):
    if sigma is not None:
        try:
            sigma = list(sigma)
        except TypeError:
            sigma = [sigma]*len(suffixes)
        sigma = dict(zip(suffixes, sigma))
    for approx in suffixes:
        display("Approximation: "+(approx if len(approx) > 0 else "tot"))
        report, sr, autosigma = getSumrule(pname, prefix, cfgname, approx)
        display(sr)
        sigma[approx] = autosigma if sigma[approx] == 0 else sigma[approx]
        display(sigma[approx])
        display(report)
        
        tab = Tabular("|l|l|c|c|c|c|c|c|")
        
        # Header
        tab.add_hline()
        tab.add_row(["", ""] + [NoEscape(i) for i in report.columns])
        tab.add_row([NoEscape(i) for i in report.index.names] + [""]*6)
        tab.add_hline()
        
        curN = "1"
        tot = report.iloc[0, 1::2].fillna(0)*0
        getN = re.compile("\((\d+)[A-Z]\)\$$")
        for entry in report.index.values:
            # Manage subtotals
            label = entry[0] if entry[0] != "${}$".format(code2name(pname)) else entry[1]
            newN = getN.search(label).groups()[0]
            if newN != curN:
                tab.add_hline()
                tab.add_hline()
                try:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data=NoEscape("${:.2f} ({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
                except:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
                tab.add_hline()
                tab.add_hline()
                tot = report.loc[entry].fillna(0).values[1::2]
                curN = newN
            else:
                tot += report.loc[entry].fillna(0).values[1::2]
            
            # Regular entry
            tab.add_row([NoEscape(i) for i in list(entry) + ["{:.2f}".format(num) for num in report.loc[entry]]])
            
        # Final subtotal
        tab.add_hline()
        tab.add_hline()
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
        tab.add_hline()
        tab.add_hline()
        
        # Total
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in sr.iloc[:, 0]])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in sr.iloc[:, 0]])                
        tab.add_hline()
        
        # with open("export/sr{}.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(sr.to_latex(escape=False, bold_rows=True, column_format="l|c"))
        # with open("export/sr{}-report.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(report.to_latex(escape=False, bold_rows=True, column_format="l|l|c|c|c|c|c|c"))
        tab.generate_tex("export/sr{}-report.{}.{}".format(approx, pname, cfgname))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [43]:
reportSumrules("yps_1S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.955
$SR-Deng$,-1.53
$SR-\Gamma$,-0.955


0.2499320450052078

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.837,0.01,-0.505,0.0166,-0.837
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.62,27.5,-1.77,25.2,-1.62
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.58,31.9,-2.54,32.4,-2.58
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.93,31.8,3.25,38.4,3.93
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0397,5.54,-0.0519,4.23,-0.0397
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.147,10.8,-0.151,10.5,-0.147
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.295,12.5,0.221,16.7,0.295
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00561,1.87,-0.00763,1.38,-0.00561
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.036,6.41,-0.0409,5.63,-0.036


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.837,0.01,-0.505,0.0166,-0.837
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.76,27.5,-1.77,27.4,-1.76
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.58,31.9,-2.43,33.9,-2.58
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.09,31.8,3.55,36.6,4.09
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0463,5.54,-0.0519,4.94,-0.0463
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.147,10.8,-0.139,11.4,-0.147
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.317,12.5,0.256,15.4,0.317
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00689,1.87,-0.00763,1.69,-0.00689
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.036,6.41,-0.0371,6.22,-0.036


'Approximation: ELW'

,$SR$
$SR-PDG$,-0.0635
$SR-Deng$,-0.553
$SR-\Gamma$,-0.0635


0.24966178223005345

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.8,27.5,-1.77,28,-1.8
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.66,31.9,-2.43,34.9,-2.66
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.23,31.8,3.55,37.9,4.23
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0356,5.54,-0.0519,3.8,-0.0356
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.127,10.8,-0.139,9.82,-0.127
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.284,12.5,0.256,13.8,0.284
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00414,1.87,-0.00763,1.02,-0.00414
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0282,6.41,-0.0371,4.88,-0.0282
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0747,8.17,0.0753,8.1,0.0747


### $\Upsilon(2S)$

In [44]:
reportSumrules("yps_2S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.804
$SR-Deng$,-0.979
$SR-\Gamma$,-1.19


1.7457987388094827

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                         
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344    0.0745   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93      0.92   
                $\chi_{b1}(1P)$   2.21    -5.77  2.17     -5.68      1.45   
                $\chi_{b2}(1P)$   2.29     9.96  2.62      11.4      1.56   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -5.42  14.4     -6.23      12.5   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.65  15.3     -7.47      15.7   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan     11.2  15.3      9.84      17.4   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  -0.0924  2.55    -0.102      2.31   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   -0.303  5.63    -0.314      5.44   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.576  6.72      0.49      7.89   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00388  
                $\chi_{b0}(1P)$       -2.47  
                $\chi_{b1}(1P)$       -3.79  
                $\chi_{b2}(1P)$         6.8  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.42  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.65  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.2  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0924  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.303  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.576

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344       0.0745   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93        0.892   
                $\chi_{b1}(1P)$   2.21    -5.84  2.17     -5.74         1.43   
                $\chi_{b2}(1P)$   2.29     9.79  2.62      11.2         1.58   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -5.66  14.4     -6.23         13.1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.65  15.3     -7.29         16.1   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan     11.4  15.3      10.3         16.9   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan   -0.102  2.55    -0.102         2.54   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   -0.303  5.63    -0.298         5.72   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.602  6.72     0.539          7.5   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00388  
                $\chi_{b0}(1P)$        -2.4  
                $\chi_{b1}(1P)$       -3.79  
                $\chi_{b2}(1P)$        6.75  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.66  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.65  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.4  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.102  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.303  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.602

'Approximation: ELW'

,$SR$
$SR-PDG$,-0.961
$SR-Deng$,-0.526
$SR-\Gamma$,-1.09


1.8269418042357826

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.27  1.09     -2.93         0.899   
                $\chi_{b1}(1P)$ 2.21    -5.84  2.17     -5.74          1.44   
                $\chi_{b2}(1P)$ 2.29     9.79  2.62      11.2          1.58   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan     -5.8  14.4     -6.23          13.4   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan    -7.92  15.3     -7.29          16.6   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.9  15.3      10.3          17.6   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan   -0.068  2.55    -0.102           1.7   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan    -0.24  5.63    -0.298          4.53   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.502  6.72     0.539          6.26   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.42  
                $\chi_{b1}(1P)$       -3.81  
                $\chi_{b2}(1P)$        6.77  
$\chi_{b0}(2P)$ $\Upsilon(2S)$         -5.8  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.92  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.9  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.068  
$\chi_{b1}(3P)$ $\Upsilon(2S)$        -0.24  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.502

### $\Upsilon(3S)$

In [45]:
reportSumrules("yps_3S", "scr", "b-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,-7.5
$SR-Deng$,-2.59
$SR-\Gamma$,-8.11


2.9834236516057175

PDG  $SR-PDG$   Deng $SR-Deng$  $\Gamma$  \
In              Out                                                           
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108    0.0788   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987    0.0994   
                $\chi_{b1}(1P)$ 0.0183  -0.00109 0.0005 -2.98e-05  8.09e-05   
                $\chi_{b2}(1P)$  0.201    0.0153   0.14    0.0107     0.149   
                $\eta_{b}(2S)$     nan  -0.00318  0.011  -0.00296    0.0118   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69      1.31   
                $\chi_{b1}(2P)$   2.56     -14.9   2.61     -15.2       2.2   
                $\chi_{b2}(2P)$   2.66      24.3   3.16      28.9      2.44   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan       -15   7.95     -12.3      9.65   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -19.2   10.3       -16      12.3   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      24.9   10.8      19.8      13.6   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\eta_{b}(1S)$      -0.0012  
                $\chi_{b0}(1P)$     -0.0101  
                $\chi_{b1}(1P)$   -4.83e-06  
                $\chi_{b2}(1P)$      0.0114  
                $\eta_{b}(2S)$     -0.00318  
                $\chi_{b0}(2P)$       -8.33  
                $\chi_{b1}(2P)$       -12.8  
                $\chi_{b2}(2P)$        22.3  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          -15  
$\chi_{b1}(3P)$ $\Upsilon(3S)$        -19.2  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         24.9

'Approximation: E1'

...

PDG  $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                 
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183  -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201    0.0142   0.14   0.00991   
                $\eta_{b}(2S)$     nan  -0.00318  0.011  -0.00296   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56     -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66        24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     -15.4   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -19.2   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      25.3   10.8      20.5   

                                 E1-$\Gamma$ $SR-\Gamma$  
In              Out                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0788     -0.0012  
                $\chi_{b0}(1P)$       0.0905    -0.00921  
                $\chi_{b1}(1P)$     7.73e-05   -4.83e-06  
                $\chi_{b2}(1P)$        0.156       0.011  
                $\eta_{b}(2S)$        0.0118    -0.00318  
                $\chi_{b0}(2P)$         1.28       -8.12  
                $\chi_{b1}(2P)$         2.18       -12.8  
                $\chi_{b2}(2P)$         2.47        22.2  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          9.96       -15.4  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          12.6       -19.2  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          13.3        25.3

'Approximation: ELW'

,$SR$
$SR-PDG$,-8.04
$SR-Deng$,-2.17
$SR-\Gamma$,-8.14


3.6313749127400636

PDG $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14   0.00991   
                $\chi_{b0}(2P)$    1.2    -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56    -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66       24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -15.9   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan      -20   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     26.5   10.8      20.5   

                                 ELW-$\Gamma$ $SR-\Gamma$  
In              Out                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.0392    -0.00399  
                $\chi_{b1}(1P)$        0.0357    -0.00223  
                $\chi_{b2}(1P)$         0.394      0.0278  
                $\chi_{b0}(2P)$           1.3       -8.23  
                $\chi_{b1}(2P)$           2.2       -12.9  
                $\chi_{b2}(2P)$          2.48        22.3  
$\chi_{b0}(3P)$ $\Upsilon(3S)$           10.2       -15.9  
$\chi_{b1}(3P)$ $\Upsilon(3S)$             13         -20  
$\chi_{b2}(3P)$ $\Upsilon(3S)$           13.9        26.5

## Charmonium sumrules

### $\psi(1S)$

In [46]:
reportSumrules("psi_1S", "scr", "c-scr", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-30.8
$SR-Deng$,-54.9
$SR-\Gamma$,-33.1


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,179,-24.7,144,-19.9
$\chi_{c1}(1P)$,$\psi(1S)$,288,-32.1,319,-35.6,331,-36.9
$\chi_{c2}(1P)$,$\psi(1S)$,367,33.5,292,26.7,501,45.8
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0193,2.3,-0.027,1.64,-0.0193
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.9,88,-1.88,88.8,-1.9
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.48,93,1.31,247,3.48


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,179,-24.7,179,-24.8
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,378,-36.9
$\chi_{c2}(1P)$,$\psi(1S)$,367,44.4,292,35.4,434,52.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0314,2.3,-0.027,2.68,-0.0314
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.9,88,-1.5,112,-1.9
$\chi_{c2}(2P)$,$\psi(1S)$,nan,4.55,93,2.18,194,4.55


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.87
$SR-Deng$,-19.8
$SR-\Gamma$,-5.57


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,179,-24.7,187,-25.8
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,411,-40.1
$\chi_{c2}(1P)$,$\psi(1S)$,367,44.4,292,35.4,481,58.3
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.037,2.3,-0.027,3.15,-0.037
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.02,88,-1.5,60.1,-1.02
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.2,93,2.18,136,3.2


In [47]:
reportSumrules("psi_1S", "lin", "c-lin", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-29.7
$SR-Deng$,-52
$SR-\Gamma$,-29.4


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,172,-23.8,140,-19.4
$\chi_{c1}(1P)$,$\psi(1S)$,288,-31.9,306,-33.8,311,-34.3
$\chi_{c2}(1P)$,$\psi(1S)$,367,34,284,26.4,477,44.2
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0929,2.64,-0.0809,3.03,-0.0929
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0339,6.1,-0.0716,2.89,-0.0339
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.93,81,-1.72,90.9,-1.93
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.86,93,1.33,270,3.86


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,172,-23.8,172,-23.8
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,352,-34.3
$\chi_{c2}(1P)$,$\psi(1S)$,367,44.4,284,34.4,416,50.3
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0929,2.64,-0.0809,3.03,-0.0929
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0544,6.1,-0.0716,4.63,-0.0544
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.93,81,-1.38,114,-1.93
$\chi_{c2}(2P)$,$\psi(1S)$,nan,5,93,2.18,213,5


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.58
$SR-Deng$,-18.5
$SR-\Gamma$,-3.97


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,147,-20.3,172,-23.8,180,-24.8
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,380,-37.1
$\chi_{c2}(1P)$,$\psi(1S)$,367,44.4,284,34.4,458,55.5
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0201,6.1,-0.0716,1.71,-0.0201
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.986,81,-1.38,58,-0.986
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.43,93,2.18,146,3.43


### $\psi(2S)$

In [48]:
reportSumrules("psi_2S", "scr", "c-scr", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-106
$SR-Deng$,-88
$SR-\Gamma$,-79.2


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0452   7.8    -0.351      10.3   
                $\chi_{c0}(1P)$    29    -18.7    22     -14.2      33.1   
                $\chi_{c1}(1P)$  28.9    -31.4    45       -49      40.9   
                $\chi_{c2}(1P)$  28.2     84.2    46       137      40.6   
                $\eta_{c}(2S)$  0.207    -21.4  0.19     -19.6     0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -74.9    99     -74.9      99.1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -202   155      -163       193   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      155   150      95.8       242   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.339   9.1    -0.425      7.25   
$\chi_{c1}(3P)$ $\psi(2S)$        nan     -4.5    74     -4.66      71.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     8.19    76      4.39       142   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.461  
                $\chi_{c0}(1P)$       -21.4  
                $\chi_{c1}(1P)$       -44.5  
                $\chi_{c2}(1P)$         121  
                $\eta_{c}(2S)$        -15.3  
$\chi_{c0}(2P)$ $\psi(2S)$            -74.9  
$\chi_{c1}(2P)$ $\psi(2S)$             -202  
$\chi_{c2}(2P)$ $\psi(2S)$              155  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.339  
$\chi_{c1}(3P)$ $\psi(2S)$             -4.5  
$\chi_{c2}(3P)$ $\psi(2S)$             8.19

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0452   7.8    -0.351         10.3   
                $\chi_{c0}(1P)$    29    -18.7    22     -14.2         28.1   
                $\chi_{c1}(1P)$  28.9    -33.1    45     -51.6         38.8   
                $\chi_{c2}(1P)$  28.2     78.1    46       128         42.4   
                $\eta_{c}(2S)$  0.207    -21.4  0.19     -19.6        0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -84.4    99     -74.9          112   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -202   155      -150          208   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      167   150       113          222   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.466   9.1    -0.425         9.98   
$\chi_{c1}(3P)$ $\psi(2S)$        nan     -4.5    74     -3.97         83.8   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     9.69    76      6.14          120   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.461  
                $\chi_{c0}(1P)$       -18.1  
                $\chi_{c1}(1P)$       -44.5  
                $\chi_{c2}(1P)$         117  
                $\eta_{c}(2S)$        -15.3  
$\chi_{c0}(2P)$ $\psi(2S)$            -84.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -202  
$\chi_{c2}(2P)$ $\psi(2S)$              167  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.466  
$\chi_{c1}(3P)$ $\psi(2S)$             -4.5  
$\chi_{c2}(3P)$ $\psi(2S)$             9.69

'Approximation: ELW'

,$SR$
$SR-PDG$,-90.8
$SR-Deng$,-48.9
$SR-\Gamma$,-62.8


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$   29    -18.7    22     -14.2          30.3   
                $\chi_{c1}(1P)$ 28.9    -33.1    45     -51.6          39.9   
                $\chi_{c2}(1P)$ 28.2     78.1    46       128          43.2   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -88.4    99     -74.9           117   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -224   155      -150           231   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      191   150       113           254   
$\chi_{c0}(3P)$ $\psi(2S)$       nan -0.00193   9.1    -0.425        0.0413   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -2.24    74     -3.97          41.7   
$\chi_{c2}(3P)$ $\psi(2S)$       nan      6.3    76      6.14            78   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -19.6  
                $\chi_{c1}(1P)$       -45.8  
                $\chi_{c2}(1P)$         120  
$\chi_{c0}(2P)$ $\psi(2S)$            -88.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -224  
$\chi_{c2}(2P)$ $\psi(2S)$              191  
$\chi_{c0}(3P)$ $\psi(2S)$         -0.00193  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.24  
$\chi_{c2}(3P)$ $\psi(2S)$              6.3

In [49]:
reportSumrules("psi_2S", "lin", "c-lin", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-128
$SR-Deng$,-120
$SR-\Gamma$,-108


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0452  8.08    -0.363      9.63   
                $\chi_{c0}(1P)$    29    -18.7    22     -14.2      33.5   
                $\chi_{c1}(1P)$  28.9    -31.4    42     -45.7      41.9   
                $\chi_{c2}(1P)$  28.2       84    38       113      38.9   
                $\eta_{c}(2S)$  0.207    -21.4  0.19     -19.6     0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -90.6   121     -91.5       120   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -235   139      -146       223   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      183   135      85.4       288   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.329    17    -0.493      11.3   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.62    94     -3.75      90.7   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     6.83    97       3.3       201   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -21.6  
                $\chi_{c1}(1P)$       -45.6  
                $\chi_{c2}(1P)$         116  
                $\eta_{c}(2S)$        -16.3  
$\chi_{c0}(2P)$ $\psi(2S)$            -90.6  
$\chi_{c1}(2P)$ $\psi(2S)$             -235  
$\chi_{c2}(2P)$ $\psi(2S)$              183  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.329  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.62  
$\chi_{c2}(3P)$ $\psi(2S)$             6.83

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0452  8.08    -0.363         9.63   
                $\chi_{c0}(1P)$    29    -18.7    22     -14.2         28.6   
                $\chi_{c1}(1P)$  28.9    -33.1    42     -48.2         39.8   
                $\chi_{c2}(1P)$  28.2     78.1    38       105         40.5   
                $\eta_{c}(2S)$  0.207    -21.4  0.19     -19.6        0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan     -103   121     -91.5          136   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -235   139      -135          242   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      198   135       102          263   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.469    17    -0.493         16.2   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.62    94     -3.15          108   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     8.25    97      4.81          166   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -18.5  
                $\chi_{c1}(1P)$       -45.6  
                $\chi_{c2}(1P)$         112  
                $\eta_{c}(2S)$        -16.3  
$\chi_{c0}(2P)$ $\psi(2S)$             -103  
$\chi_{c1}(2P)$ $\psi(2S)$             -235  
$\chi_{c2}(2P)$ $\psi(2S)$              198  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.469  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.62  
$\chi_{c2}(3P)$ $\psi(2S)$             8.25

'Approximation: ELW'

,$SR$
$SR-PDG$,-112
$SR-Deng$,-80.6
$SR-\Gamma$,-90.8


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$   29    -18.7    22     -14.2          30.7   
                $\chi_{c1}(1P)$ 28.9    -33.1    42     -48.2          40.9   
                $\chi_{c2}(1P)$ 28.2     78.1    38       105          41.2   
$\chi_{c0}(2P)$ $\psi(2S)$       nan     -108   121     -91.5           143   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -261   139      -135           269   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      228   135       102           303   
$\chi_{c0}(3P)$ $\psi(2S)$       nan   -0.014    17    -0.493         0.484   
$\chi_{c1}(3P)$ $\psi(2S)$       nan     -1.3    94     -3.15          38.8   
$\chi_{c2}(3P)$ $\psi(2S)$       nan     4.44    97      4.81          89.6   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -19.8  
                $\chi_{c1}(1P)$       -46.9  
                $\chi_{c2}(1P)$         114  
$\chi_{c0}(2P)$ $\psi(2S)$             -108  
$\chi_{c1}(2P)$ $\psi(2S)$             -261  
$\chi_{c2}(2P)$ $\psi(2S)$              228  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.014  
$\chi_{c1}(3P)$ $\psi(2S)$             -1.3  
$\chi_{c2}(3P)$ $\psi(2S)$             4.44

### $\psi(3S)$

In [50]:
reportSumrules("psi_3S", "scr", "c-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,65.5
$SR-Deng$,203
$SR-\Gamma$,65.5


58.02581458868679

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.144  6.76     -0.101      9.62   
           $\chi_{c0}(1P)$  nan     -1.16   6.7     -0.404      19.2   
           $\chi_{c1}(1P)$  nan      -0.4   6.7     -0.274      9.78   
           $\chi_{c2}(1P)$  nan     0.152   2.5      0.196      1.93   
           $\eta_{c}(2S)$   nan    -0.417  1.78      -0.37      2.01   
           $\chi_{c0}(2P)$  nan     -71.3    27        -60      32.1   
           $\chi_{c1}(2P)$  nan     -57.6    67      -84.4      45.7   
           $\chi_{c2}(2P)$  nan       206    82        373      45.2   
           $\eta_{c}(3S)$   nan     -9.55 0.088      -24.5    0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.144  
           $\chi_{c0}(1P)$        -1.16  
           $\chi_{c1}(1P)$         -0.4  
           $\chi_{c2}(1P)$        0.152  
           $\eta_{c}(2S)$        -0.417  
           $\chi_{c0}(2P)$        -71.3  
           $\chi_{c1}(2P)$        -57.6  
           $\chi_{c2}(2P)$          206  
           $\eta_{c}(3S)$         -9.55

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.144  6.76     -0.101         9.62   
           $\chi_{c0}(1P)$  nan    -0.824   6.7     -0.404         13.7   
           $\chi_{c1}(1P)$  nan      -0.4   6.7     -0.321         8.35   
           $\chi_{c2}(1P)$  nan     0.138   2.5      0.154         2.23   
           $\eta_{c}(2S)$   nan    -0.417  1.78      -0.37         2.01   
           $\chi_{c0}(2P)$  nan     -63.3    27        -60         28.5   
           $\chi_{c1}(2P)$  nan     -57.6    67      -87.8         43.9   
           $\chi_{c2}(2P)$  nan       201    82        353         46.7   
           $\eta_{c}(3S)$   nan     -9.55 0.088      -24.5       0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.144  
           $\chi_{c0}(1P)$       -0.824  
           $\chi_{c1}(1P)$         -0.4  
           $\chi_{c2}(1P)$        0.138  
           $\eta_{c}(2S)$        -0.417  
           $\chi_{c0}(2P)$        -63.3  
           $\chi_{c1}(2P)$        -57.6  
           $\chi_{c2}(2P)$          201  
           $\eta_{c}(3S)$         -9.55

'Approximation: ELW'

,$SR$
$SR-PDG$,75.9
$SR-Deng$,204
$SR-\Gamma$,75.9


62.75474765875331

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.466   6.7     -0.404          7.73   
           $\chi_{c1}(1P)$  nan   -0.0323   6.7     -0.321         0.675   
           $\chi_{c2}(1P)$  nan     0.119   2.5      0.154          1.92   
           $\chi_{c0}(2P)$  nan     -69.7    27        -60          31.4   
           $\chi_{c1}(2P)$  nan     -59.9    67      -87.8          45.7   
           $\chi_{c2}(2P)$  nan       206    82        353          47.9   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.466  
           $\chi_{c1}(1P)$      -0.0323  
           $\chi_{c2}(1P)$        0.119  
           $\chi_{c0}(2P)$        -69.7  
           $\chi_{c1}(2P)$        -59.9  
           $\chi_{c2}(2P)$          206

In [51]:
reportSumrules("psi_3S", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,1.03e+03
$SR-Deng$,1.09e+03
$SR-\Gamma$,1.03e+03


21.727959166988295

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0995      10.4   
           $\chi_{c0}(1P)$  nan     -1.31   5.9     -0.356      21.6   
           $\chi_{c1}(1P)$  nan    -0.409     4     -0.163      10.1   
           $\chi_{c2}(1P)$  nan    0.0613  0.25     0.0198     0.775   
           $\eta_{c}(2S)$   nan    -0.582  1.65     -0.343       2.8   
           $\chi_{c0}(2P)$  nan     -95.4    19      -42.2      42.9   
           $\chi_{c1}(2P)$  nan     -81.8    55      -68.9      65.3   
           $\chi_{c2}(2P)$  nan       291    67        307      63.4   
           $\eta_{c}(3S)$   nan       920 0.051        891    0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$        -1.31  
           $\chi_{c1}(1P)$       -0.409  
           $\chi_{c2}(1P)$       0.0613  
           $\eta_{c}(2S)$        -0.582  
           $\chi_{c0}(2P)$        -95.4  
           $\chi_{c1}(2P)$        -81.8  
           $\chi_{c2}(2P)$          291  
           $\eta_{c}(3S)$           920

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0995         10.4   
           $\chi_{c0}(1P)$  nan    -0.925   5.9     -0.356         15.3   
           $\chi_{c1}(1P)$  nan    -0.409     4     -0.192         8.54   
           $\chi_{c2}(1P)$  nan    0.0554  0.25     0.0154          0.9   
           $\eta_{c}(2S)$   nan    -0.582  1.65     -0.343          2.8   
           $\chi_{c0}(2P)$  nan     -83.9    19      -42.2         37.8   
           $\chi_{c1}(2P)$  nan     -81.8    55      -72.1         62.4   
           $\chi_{c2}(2P)$  nan       283    67        288         65.7   
           $\eta_{c}(3S)$   nan       920 0.051        891       0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$       -0.925  
           $\chi_{c1}(1P)$       -0.409  
           $\chi_{c2}(1P)$       0.0554  
           $\eta_{c}(2S)$        -0.582  
           $\chi_{c0}(2P)$        -83.9  
           $\chi_{c1}(2P)$        -81.8  
           $\chi_{c2}(2P)$          283  
           $\eta_{c}(3S)$           920

'Approximation: ELW'

,$SR$
$SR-PDG$,112
$SR-Deng$,173
$SR-\Gamma$,112


20.835134470925418

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan     -0.44   5.9     -0.356           7.3   
           $\chi_{c1}(1P)$  nan  -0.00502     4     -0.192         0.105   
           $\chi_{c2}(1P)$  nan     0.437  0.25     0.0154           7.1   
           $\chi_{c0}(2P)$  nan     -93.2    19      -42.2          41.9   
           $\chi_{c1}(2P)$  nan     -85.7    55      -72.1          65.4   
           $\chi_{c2}(2P)$  nan       291    67        288          67.7   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$        -0.44  
           $\chi_{c1}(1P)$     -0.00502  
           $\chi_{c2}(1P)$        0.437  
           $\chi_{c0}(2P)$        -93.2  
           $\chi_{c1}(2P)$        -85.7  
           $\chi_{c2}(2P)$          291

In [52]:
reportSumrules("psi_1_1D", "scr", "c-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-209
$SR-Deng$,-267
$SR-\Gamma$,-251


11.780664938953366

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.0137   nan   -0.0137     0.427   
                $\chi_{c0}(1P)$  188    -52.3   261     -72.7       275   
                $\chi_{c1}(1P)$ 67.7    -24.5   135     -48.8       119   
                $\chi_{c2}(1P)$  nan     1.84   8.1      2.59      5.74   
                $\eta_{c}(2S)$   nan -0.00094   nan  -0.00094  0.000214   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -48.6    12     -80.7      7.22   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -87.3   9.8     -69.2      12.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.63  0.46      1.62     0.747   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.0551  0.39   -0.0343     0.627   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   -0.305     2    -0.168      3.63   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  -0.0428  0.79   -0.0268      1.26   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0137  
                $\chi_{c0}(1P)$       -76.6  
                $\chi_{c1}(1P)$         -43  
                $\chi_{c2}(1P)$        1.84  
                $\eta_{c}(2S)$     -0.00094  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -48.6  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -87.3  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.63  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0551  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.305  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0428

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.0137   nan   -0.0137        0.427   
                $\chi_{c0}(1P)$  188    -52.3   261     -72.7          311   
                $\chi_{c1}(1P)$ 67.7    -22.7   135     -45.3          143   
                $\chi_{c2}(1P)$  nan     4.48   8.1      4.67         7.76   
                $\eta_{c}(2S)$   nan -0.00094   nan  -0.00094     0.000214   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -47.6    12     -80.7         7.08   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -83.1   9.8     -71.5         11.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.86  0.46      1.41        0.607   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.0493  0.39   -0.0343         0.56   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   -0.269     2    -0.189         2.84   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   0.0303  0.79      0.11        0.218   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0137  
                $\chi_{c0}(1P)$       -86.7  
                $\chi_{c1}(1P)$       -48.1  
                $\chi_{c2}(1P)$        4.48  
                $\eta_{c}(2S)$     -0.00094  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -47.6  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -83.1  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.86  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0493  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.269  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0303

'Approximation: ELW'

,$SR$
$SR-PDG$,-203
$SR-Deng$,-264
$SR-\Gamma$,-276


15.412251164113922

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   261     -72.7           349   
                $\chi_{c1}(1P)$ 67.7    -22.7   135     -45.3           153   
                $\chi_{c2}(1P)$  nan     4.74   8.1      4.67          8.22   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -47.9    12     -80.7          7.12   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -85.8   9.8     -71.5          11.8   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.96  0.46      1.41         0.638   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.746  0.39   -0.0343          8.48   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0335     2    -0.189         0.355   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     0.01  0.79      0.11        0.0719   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -97.1  
                $\chi_{c1}(1P)$       -51.5  
                $\chi_{c2}(1P)$        4.74  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -47.9  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -85.8  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.96  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.746  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0335  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.01

In [53]:
reportSumrules("psi_1_1D", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,-319
$SR-Deng$,-312
$SR-\Gamma$,-360


23.395043370794568

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0117   nan   -0.0117     0.365   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8       273   
                $\chi_{c1}(1P)$ 67.7     -24.4   138     -49.7       115   
                $\chi_{c2}(1P)$  nan      1.83   7.1      2.46      5.28   
                $\eta_{c}(2S)$   nan -0.000608   nan -0.000608  0.000138   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -119    20      -134      17.7   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan      -129   7.9     -55.5      18.3   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      3.88  0.36      1.28      1.09   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00612  0.27   -0.0132     0.126   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.311   3.2    -0.148      6.71   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0698   1.5   -0.0387       2.7   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0117  
                $\chi_{c0}(1P)$         -76  
                $\chi_{c1}(1P)$       -41.5  
                $\chi_{c2}(1P)$        1.83  
                $\eta_{c}(2S)$    -0.000608  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -119  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$         -129  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         3.88  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00612  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.311  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0698

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0117   nan   -0.0117        0.365   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8          307   
                $\chi_{c1}(1P)$ 67.7     -22.7   138     -46.3          137   
                $\chi_{c2}(1P)$  nan      4.06   7.1      4.09         7.03   
                $\eta_{c}(2S)$   nan -0.000608   nan -0.000608     0.000138   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -116    20      -134         17.2   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan      -121   7.9     -57.6         16.7   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.61  0.36      1.11        0.849   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00538  0.27   -0.0132         0.11   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -0.27   3.2    -0.171         5.07   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0281   1.5     0.116        0.363   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0117  
                $\chi_{c0}(1P)$       -85.4  
                $\chi_{c1}(1P)$       -46.1  
                $\chi_{c2}(1P)$        4.06  
                $\eta_{c}(2S)$    -0.000608  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -116  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$         -121  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.61  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00538  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -0.27  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0281

'Approximation: ELW'

,$SR$
$SR-PDG$,-312
$SR-Deng$,-309
$SR-\Gamma$,-381


24.692742701319766

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   272     -75.8           339   
                $\chi_{c1}(1P)$ 67.7    -22.7   138     -46.3           146   
                $\chi_{c2}(1P)$  nan     4.26   7.1      4.09          7.39   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -117    20      -134          17.4   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -127   7.9     -57.6          17.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.79  0.36      1.11         0.908   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.427  0.27   -0.0132          8.75   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0175   3.2    -0.171         0.328   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.00675   1.5     0.116        0.0873   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -94.5  
                $\chi_{c1}(1P)$         -49  
                $\chi_{c2}(1P)$        4.26  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -117  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$         -127  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.79  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.427  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0175  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      0.00675

## Sumrules for the article

In [54]:
def getWidths(pname, prefix, cfgname):
    my_full = getRs(pname, prefix, cfgname, suffix="")
    my_full.rename(columns={"$\Gamma$": "FULL", "$r^{(0)}$": "r0", "$r^{(2)}$": "r2"}, inplace=True)
    my_lwa = getRs(pname, prefix, cfgname, suffix="ELW")
    my_lwa.rename(columns={"$\Gamma$": "LWA", "$r^{(0)}$": "r0lwa", "$r^{(2)}$": "r2lwa"}, inplace=True)
    deng = pd.concat([getDengTrans(prefix, cfg) for\
                          cfg in data_pack[cfgname.split("-")[0]]
                     ], sort=False, axis=0)
    pdg, pdg_err = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(my_full.index)
    pdg_err = pdg_err.reindex(my_full.index)
    deng = deng.reindex(my_full.index.values)
    res = pd.concat([pdg, my_full, my_lwa, deng], axis=1)
    err = res*sp.nan
    err["PDG"] = pdg_err
    return res, err.abs()


In [55]:
def getSumrule(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    
    widths, widths_err = getWidths(pname, prefix, cfgname)
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule_prefactors = []
    
    abovethr = widths.query("Out == r'{}'".format(pretty_pname))
    abovethr_pcls = [p[0] for p in abovethr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[abovethr_pcls, "E"]**3\
        /(masses.loc[abovethr_pcls, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr_pcls, "xJ"]
    )
    sumrule_prefactors[0].index = pd.Index([(v, pretty_pname) for v in sumrule_prefactors[0].index.values])
    
    belowthr = widths.query("In == r'{}'".format(pretty_pname))
    belowthr_pcls = [p[1] for p in belowthr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr_pcls, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule_prefactors[1].index = pd.Index([(pretty_pname, v) for v in sumrule_prefactors[1].index.values])
    
    sumrule_prefactors_cnct = pd.concat(sumrule_prefactors, axis=0)
    sumrule_prefactors_cnct = sumrule_prefactors_cnct.reindex(widths.index.values)
    
    sumrule_prefactors_cnct = sumrule_prefactors_cnct*(widths.loc[:, "r2"] - widths.loc[:, "r0"])\
                *(0.19732697)**2*10**4*10**3
    sumrule = widths.drop(columns=["r0", "r2", "r0lwa", "r2lwa"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    sumrule_err = widths_err.drop(columns=["r0", "r2", "r0lwa", "r2lwa"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    
    widths.drop(columns=["r2", "r0", "r0lwa", "r2lwa"], inplace=True)
   
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    sumrule = sumrule.reindex(widths.index.values)
    sumrule_err = sumrule_err.reindex(widths.index.values) 
    
    sumrule = pd.concat([sumrule.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,sumrule.drop(columns=["PDG", "FULL", "LWA", "Deng"]).mean(axis=1).rename("E1av")], axis=1)
    sumrule_err = pd.concat([sumrule_err.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,sumrule_err.drop(columns=["PDG", "FULL", "LWA", "Deng"]).mean(axis=1).rename("E1av")], axis=1)
    widths = pd.concat([widths.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,widths.drop(columns=["PDG", "FULL", "LWA", "Deng"]).mean(axis=1).rename("E1av")], axis=1)
    widths_err = pd.concat([widths_err.loc[:, ["PDG", "FULL", "LWA", "Deng"]]
                         ,widths_err.drop(columns=["PDG", "FULL", "LWA", "Deng"]).mean(axis=1).rename("E1av")], axis=1)
    
    masses = dfsort(\
                      masses.loc[belowthr_pcls+abovethr_pcls, :]
                    , lambda a, b: \
                        cmpStatesByName(prefix, cfgname, a[1:-1], b[1:-1]) \
                   ).rename(columns={"E": "M"})
    
    return masses, sumrule, sumrule_err.abs(), widths, widths_err

In [56]:
def sumruleReport(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    def numfmt(num):
        return "{:.3g}".format(num)
    
    masses, sumrule, sumrule_err, widths, widths_err = getSumrule(pname, prefix, cfgname)
    abovethr = [pair[0] for pair in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    
    tab = LongTable("|c|c|c|c|c|")

    # Header
    tab.add_hline()
    tab.add_row([  "X"
                 , NoEscape(r"$m_X$ [$MeV$]")\
                 , NoEscape(r"$\Gamma({} \rightarrow \gamma X )$".format(code2name(pname)))\
                 , NoEscape(r"$\Gamma(X \rightarrow \gamma {})$".format(code2name(pname)))\
                 , NoEscape(r"$SR_1$ [$\mu b$]")
                ])
    tab.add_hline()

    curN = "1"
    tot = sumrule.iloc[0].fillna(0)*0
    tot_err = sumrule_err.iloc[0].fillna(0)*0
    getN = re.compile("\((\d+)[A-Z]\)\$$")
    for label in masses.index.values:
        if label in abovethr:
            label_pair = (label, pretty_pname)
        else:
            label_pair = (pretty_pname, label)

        # Manage subtotals
        newN = getN.search(label).groups()[0]
        if newN != curN:
            tab.add_hline()
            first = True
            for source in tot.index.values:
                if tot[source] == 0:
                    continue
                sr_err = ""
                if tot_err[source] != 0:
                    sr_err = "$\pm$"+numfmt(sp.sqrt(tot_err[source]))
                if first:
                    tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
                else:
                    tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
                if first:
                    first = False
            tab.add_hline()
            tab.add_hline()
            tot = sumrule.loc[label_pair].fillna(0)
            tot_err = sumrule_err.loc[label_pair].fillna(0)**2
            curN = newN
        else:
            tot += sumrule.loc[label_pair].fillna(0)
            tot_err += sumrule_err.loc[label_pair].fillna(0)**2
            
        # Regular entry
        first=True
        curdata = widths.loc[label_pair, :]
        for source in curdata.index.values:
            sr_err = ""
            if not sp.isnan(sumrule_err.loc[label_pair, source]):
                sr_err = "$\pm$"+numfmt(sumrule_err.loc[label_pair, source])
            srentry =  sumrule.loc[label_pair][source]
            if sp.isnan(srentry) or srentry == 0:
                continue
            
            dw_err = ""
            if not sp.isnan(widths_err.loc[label_pair, source]):
                dw_err = "$\pm$"+numfmt(widths_err.loc[label_pair, source])
            dw = numfmt(widths.loc[label_pair, source])+dw_err+" [{}]".format(source)
            dwentry = (dw, "") if label in abovethr else ("", dw)
            
            if first:
                tab.add_row([NoEscape(i) for i in [label, numfmt(masses.loc[label, "M"]), *dwentry, numfmt(srentry)+sr_err]])
            else:
                tab.add_row([NoEscape(i) for i in ["", "", *dwentry, numfmt(srentry)+sr_err]])
            if first:
                first = False
        tab.add_hline()

    # Final subtotal
    tab.add_hline()
    first = True
    for source in tot.index.values:
        if tot[source] == 0:
            continue
        sr_err = ""
        if tot_err[source] != 0:
            sr_err = "$\pm$"+numfmt(tot_err[source])
        if first:
            tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        else:
            tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        if first:
            first = False
    tab.add_hline()
    tab.add_hline()

    # Total
    tot = sumrule.sum(axis=0)
    tot_err = sumrule_err.apply(sp.square).sum(axis=0).apply(sp.sqrt)
    first = True
    for source in tot.index.values:
        if tot[source] == 0:
            continue
        sr_err = ""
        if tot_err[source] != 0:
            sr_err = "$\pm$"+numfmt(tot_err[source])        
        if first:
            tab.add_row([NoEscape(i) for i in ["Total"] + [""]*3 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        else:
            tab.add_row([NoEscape(i) for i in [""]*4 + [numfmt(tot[source])+sr_err+" [{}]".format(source)]])
        if first:
            first = False
    tab.add_hline()
    
    return tab

In [57]:
def doReport(pname, prefix, cfgname):
    masses, sr, sr_err, widths, widths_err = getSumrule(pname, prefix, cfgname)
    display(sr.sum(axis=0))
    display(widths)
    display(sr)
    sumruleReport(pname, prefix, cfgname).generate_tex("export/sr-article.{}.{}".format(pname, cfgname))

### Bottomonium sumrules

In [58]:
doReport("yps_1S", "scr", "b-scr")

PDG         0
FULL   -0.955
LWA      -0.6
Deng    -1.53
E1av    -1.08
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.0166,nan,0.01,0.0101
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,25.2,28,27.5,26.5
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,32.4,34.9,31.9,33.1
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,38.4,37.9,31.8,36.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,4.23,3.8,5.54,6.13
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.5,9.82,10.8,8.62
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,16.7,13.8,12.5,10.7
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.38,1.02,1.87,2.39
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,5.63,4.88,6.41,3.92


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,NaN,-0.837,NaN,-0.505,-0.51
$\chi_{b0}(1P)$,$\Upsilon(1S)$,NaN,-1.62,-1.8,-1.77,-1.71
$\chi_{b1}(1P)$,$\Upsilon(1S)$,NaN,-2.58,-2.78,-2.54,-2.63
$\chi_{b2}(1P)$,$\Upsilon(1S)$,NaN,3.93,3.88,3.25,3.74
$\chi_{b0}(2P)$,$\Upsilon(1S)$,NaN,-0.0397,-0.0356,-0.0519,-0.0574
$\chi_{b1}(2P)$,$\Upsilon(1S)$,NaN,-0.147,-0.137,-0.151,-0.12
$\chi_{b2}(2P)$,$\Upsilon(1S)$,NaN,0.295,0.245,0.221,0.188
$\chi_{b0}(3P)$,$\Upsilon(1S)$,NaN,-0.00561,-0.00414,-0.00763,-0.00975
$\chi_{b1}(3P)$,$\Upsilon(1S)$,NaN,-0.036,-0.0312,-0.0409,-0.025


In [59]:
doReport("yps_2S", "scr", "b-scr")

PDG      0.92
FULL    -1.19
LWA     -1.72
Deng   -0.979
E1av   -0.626
dtype: float64

PDG   FULL   LWA  Deng   E1av
In              Out                                             
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.0745   nan 0.066 0.0375
                $\chi_{b0}(1P)$   1.22   0.92 0.899  1.09   1.28
                $\chi_{b1}(1P)$   2.21   1.45  1.44  2.17   2.11
                $\chi_{b2}(1P)$   2.29   1.56  1.58  2.62    2.3
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan   12.5  13.4  14.4   11.6
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan   15.7  16.6  15.3   14.7
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan   17.4  17.6  15.3   16.3
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan   2.31   1.7  2.55   3.29
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   5.44  4.53  5.63   4.64
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan   7.89  6.26  6.72   5.75

PDG     FULL    LWA     Deng     E1av
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$  -0.00065 -0.00388    NaN -0.00344 -0.00195
                $\chi_{b0}(1P)$    -3.27    -2.47  -2.42    -2.93    -3.44
                $\chi_{b1}(1P)$    -5.77    -3.79  -3.77    -5.68    -5.51
                $\chi_{b2}(1P)$     9.96      6.8   6.89     11.4       10
$\chi_{b0}(2P)$ $\Upsilon(2S)$       NaN    -5.42   -5.8    -6.23    -5.02
$\chi_{b1}(2P)$ $\Upsilon(2S)$       NaN    -7.65  -8.11    -7.47    -7.18
$\chi_{b2}(2P)$ $\Upsilon(2S)$       NaN     11.2   11.3     9.84     10.5
$\chi_{b0}(3P)$ $\Upsilon(2S)$       NaN  -0.0924 -0.068   -0.102   -0.132
$\chi_{b1}(3P)$ $\Upsilon(2S)$       NaN   -0.303 -0.253   -0.314   -0.258
$\chi_{b2}(3P)$ $\Upsilon(2S)$       NaN    0.576  0.456     0.49    0.419

In [60]:
doReport("yps_3S", "scr", "b-scr")

PDG     1.77
FULL   -8.11
LWA    -8.95
Deng   -2.59
E1av    -3.5
dtype: float64

PDG     FULL    LWA   Deng    E1av
In              Out                                                  
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104   0.0788    nan  0.071  0.0427
                $\chi_{b0}(1P)$ 0.0549   0.0994 0.0392  0.097  0.0614
                $\chi_{b1}(1P)$ 0.0183 8.09e-05 0.0357 0.0005   0.117
                $\chi_{b2}(1P)$  0.201    0.149  0.394   0.14   0.428
                $\eta_{b}(2S)$     nan   0.0118    nan  0.011 0.00423
                $\chi_{b0}(2P)$    1.2     1.31    1.3   1.21    1.36
                $\chi_{b1}(2P)$   2.56      2.2    2.2   2.61    2.37
                $\chi_{b2}(2P)$   2.66     2.44   2.48   3.16    2.76
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     9.65   10.2   7.95    7.74
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     12.3     13   10.3      10
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     13.6   13.9   10.8    11.6

PDG      FULL      LWA      Deng  \
In              Out                                                      
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000157   -0.0012      NaN  -0.00108   
                $\chi_{b0}(1P)$  -0.00558   -0.0101 -0.00399  -0.00987   
                $\chi_{b1}(1P)$  -0.00109 -4.83e-06 -0.00213 -2.98e-05   
                $\chi_{b2}(1P)$    0.0153    0.0114     0.03    0.0107   
                $\eta_{b}(2S)$        NaN  -0.00318      NaN  -0.00296   
                $\chi_{b0}(2P)$     -7.62     -8.33    -8.23     -7.69   
                $\chi_{b1}(2P)$     -14.9     -12.8    -12.8     -15.2   
                $\chi_{b2}(2P)$      24.3      22.3     22.7      28.9   
$\chi_{b0}(3P)$ $\Upsilon(3S)$        NaN       -15    -15.9     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$        NaN     -19.2    -20.3       -16   
$\chi_{b2}(3P)$ $\Upsilon(3S)$        NaN      24.9     25.6      19.8   

                                     E1av  
In              Out                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000648  
                $\chi_{b0}(1P)$  -0.00625  
                $\chi_{b1}(1P)$  -0.00701  
                $\chi_{b2}(1P)$    0.0326  
                $\eta_{b}(2S)$   -0.00114  
                $\chi_{b0}(2P)$     -8.63  
                $\chi_{b1}(2P)$     -13.8  
                $\chi_{b2}(2P)$      25.2  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -12  
$\chi_{b1}(3P)$ $\Upsilon(3S)$      -15.6  
$\chi_{b2}(3P)$ $\Upsilon(3S)$       21.3

###  Charmonium Screened Sumrules

In [61]:
doReport("psi_1S", "scr", "c-scr")

PDG    -32.3
FULL   -33.1
LWA    -27.2
Deng   -54.9
E1av     -38
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.78,nan,2.44,2.19
$\chi_{c0}(1P)$,$\psi(1S)$,147,144,187,179,141
$\chi_{c1}(1P)$,$\psi(1S)$,288,331,411,319,284
$\chi_{c2}(1P)$,$\psi(1S)$,367,501,481,292,349
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.91,nan,2.29,4.42
$\chi_{c0}(2P)$,$\psi(1S)$,nan,1.64,3.15,2.3,21.7
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88.8,60.1,88,60.9
$\chi_{c2}(2P)$,$\psi(1S)$,nan,247,136,93,96.3


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,-13.3,-23.5,NaN,-20.6,-18.5
$\chi_{c0}(1P)$,$\psi(1S)$,-20.3,-19.9,-25.8,-24.7,-19.5
$\chi_{c1}(1P)$,$\psi(1S)$,-32.1,-36.9,-45.8,-35.6,-31.6
$\chi_{c2}(1P)$,$\psi(1S)$,33.5,45.8,43.9,26.7,31.9
$\eta_{c}(2S)$,$\psi(1S)$,NaN,-0.0893,NaN,-0.0702,-0.135
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0193,-0.037,-0.027,-0.255
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.9,-1.28,-1.88,-1.3
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,3.48,1.92,1.31,1.36


In [62]:
doReport("psi_2S", "scr", "c-scr")

PDG     12.7
FULL   -79.2
LWA     -100
Deng     -88
E1av   -74.1
dtype: float64

PDG  FULL    LWA  Deng  E1av
In              Out                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  10.3    nan   7.8  5.81
                $\chi_{c0}(1P)$    29  33.1   30.3    22  36.2
                $\chi_{c1}(1P)$  28.9  40.9   39.9    45  42.4
                $\chi_{c2}(1P)$  28.2  40.6   43.2    46  34.4
                $\eta_{c}(2S)$  0.207 0.148    nan  0.19 0.153
$\chi_{c0}(2P)$ $\psi(2S)$        nan  99.1    117    99  88.9
$\chi_{c1}(2P)$ $\psi(2S)$        nan   193    231   155   135
$\chi_{c2}(2P)$ $\psi(2S)$        nan   242    254   150   183
$\chi_{c0}(3P)$ $\psi(2S)$        nan  7.25 0.0413   9.1  13.8
$\chi_{c1}(3P)$ $\psi(2S)$        nan  71.5   41.7    74  57.8
$\chi_{c2}(3P)$ $\psi(2S)$        nan   142     78    76  77.6

PDG   FULL      LWA   Deng   E1av
In              Out                                                  
$\psi(2S)$      $\eta_{c}(1S)$  -0.0452 -0.461      NaN -0.351 -0.261
                $\chi_{c0}(1P)$   -18.7  -21.4    -19.6  -14.2  -23.3
                $\chi_{c1}(1P)$   -31.4  -44.5    -43.4    -49  -46.1
                $\chi_{c2}(1P)$    84.2    121      129    137    103
                $\eta_{c}(2S)$    -21.4  -15.3      NaN  -19.6  -15.8
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -74.9    -88.4  -74.9  -67.2
$\chi_{c1}(2P)$ $\psi(2S)$          NaN   -202     -242   -163   -141
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    155      162   95.8    117
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.339 -0.00193 -0.425 -0.644
$\chi_{c1}(3P)$ $\psi(2S)$          NaN   -4.5    -2.62  -4.66  -3.64
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   8.19      4.5   4.39   4.48

In [63]:
doReport("psi_1_1D", "scr", "c-scr")

PDG    -76.8
FULL    -251
LWA     -280
Deng    -267
E1av    -378
dtype: float64

PDG     FULL    LWA  Deng  E1av
In              Out                                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    0.427    nan   nan   nan
                $\chi_{c0}(1P)$  188      275    349   261   300
                $\chi_{c1}(1P)$ 67.7      119    153   135   118
                $\chi_{c2}(1P)$  nan     5.74   8.22   8.1  5.46
                $\eta_{c}(2S)$   nan 0.000214    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     7.22   7.12    12  23.4
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     12.4   11.8   9.8  14.1
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan    0.747  0.638  0.46  0.87
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    0.627   8.48  0.39  2.15
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     3.63  0.355     2  2.14
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     1.26 0.0719  0.79  0.88

PDG     FULL      LWA    Deng    E1av
In              Out                                                    
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN  -0.0137      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3    -76.6    -97.1   -72.7   -83.6
                $\chi_{c1}(1P)$ -24.5      -43    -55.5   -48.8   -42.7
                $\chi_{c2}(1P)$   NaN     1.84     2.63    2.59    1.75
                $\eta_{c}(2S)$    NaN -0.00094      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN    -48.6    -47.9   -80.7    -157
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN    -87.3    -83.1   -69.2   -99.3
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN     2.63     2.25    1.62    3.07
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN  -0.0551   -0.746 -0.0343  -0.189
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN   -0.305  -0.0297  -0.168  -0.179
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN  -0.0428 -0.00244 -0.0268 -0.0298

### Charmonium Linear

In [64]:
doReport("psi_1S", "lin", "c-lin")

PDG    -31.5
FULL   -29.4
LWA    -23.5
Deng     -52
E1av   -37.5
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.57,nan,2.39,2.2
$\chi_{c0}(1P)$,$\psi(1S)$,147,140,180,172,142
$\chi_{c1}(1P)$,$\psi(1S)$,288,311,380,306,285
$\chi_{c2}(1P)$,$\psi(1S)$,367,477,458,284,350
$\eta_{c}(2S)$,$\psi(1S)$,nan,3.03,nan,2.64,4.33
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.89,1.71,6.1,21.2
$\chi_{c1}(2P)$,$\psi(1S)$,nan,90.9,58,81,61.9
$\chi_{c2}(2P)$,$\psi(1S)$,nan,270,146,93,96.3


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,-13.3,-21.7,NaN,-20.2,-18.6
$\chi_{c0}(1P)$,$\psi(1S)$,-20.3,-19.4,-24.8,-23.8,-19.6
$\chi_{c1}(1P)$,$\psi(1S)$,-31.9,-34.3,-42,-33.8,-31.5
$\chi_{c2}(1P)$,$\psi(1S)$,34,44.2,42.5,26.4,32.5
$\eta_{c}(2S)$,$\psi(1S)$,NaN,-0.0929,NaN,-0.0809,-0.133
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0339,-0.0201,-0.0716,-0.249
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.93,-1.23,-1.72,-1.31
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,3.86,2.09,1.33,1.37


In [65]:
doReport("psi_2S", "lin", "c-lin")

PDG     12.4
FULL    -108
LWA     -140
Deng    -120
E1av   -72.2
dtype: float64

PDG  FULL   LWA  Deng  E1av
In              Out                                          
$\psi(2S)$      $\eta_{c}(1S)$   1.01  9.63   nan  8.08  5.74
                $\chi_{c0}(1P)$    29  33.5  30.7    22  36.2
                $\chi_{c1}(1P)$  28.9  41.9  40.9    42  42.7
                $\chi_{c2}(1P)$  28.2  38.9  41.2    38  35.4
                $\eta_{c}(2S)$  0.207 0.158   nan  0.19 0.153
$\chi_{c0}(2P)$ $\psi(2S)$        nan   120   143   121  85.7
$\chi_{c1}(2P)$ $\psi(2S)$        nan   223   269   139   137
$\chi_{c2}(2P)$ $\psi(2S)$        nan   288   303   135   185
$\chi_{c0}(3P)$ $\psi(2S)$        nan  11.3 0.484    17  12.2
$\chi_{c1}(3P)$ $\psi(2S)$        nan  90.7  38.8    94  53.8
$\chi_{c2}(3P)$ $\psi(2S)$        nan   201  89.6    97  73.4

PDG   FULL    LWA   Deng   E1av
In              Out                                                
$\psi(2S)$      $\eta_{c}(1S)$  -0.0452 -0.433    NaN -0.363 -0.258
                $\chi_{c0}(1P)$   -18.7  -21.6  -19.8  -14.2  -23.3
                $\chi_{c1}(1P)$   -31.4  -45.6  -44.5  -45.7  -46.5
                $\chi_{c2}(1P)$      84    116    123    113    105
                $\eta_{c}(2S)$    -21.4  -16.3    NaN  -19.6  -15.8
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -90.6   -108  -91.5  -64.9
$\chi_{c1}(2P)$ $\psi(2S)$          NaN   -235   -283   -146   -144
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    183    192   85.4    117
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.329 -0.014 -0.493 -0.354
$\chi_{c1}(3P)$ $\psi(2S)$          NaN  -3.62  -1.55  -3.75  -2.14
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   6.83   3.05    3.3    2.5

In [66]:
doReport("psi_1_1D", "lin", "c-lin")

PDG    -76.7
FULL    -360
LWA     -381
Deng    -312
E1av    -369
dtype: float64

PDG     FULL    LWA  Deng  E1av
In              Out                                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    0.365    nan   nan   nan
                $\chi_{c0}(1P)$  188      273    339   272   299
                $\chi_{c1}(1P)$ 67.7      115    146   138   118
                $\chi_{c2}(1P)$  nan     5.28   7.39   7.1  5.59
                $\eta_{c}(2S)$   nan 0.000138    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     17.7   17.4    20  21.8
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     18.3   17.4   7.9  14.4
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.09  0.908  0.36  0.89
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    0.126   8.75  0.27  2.18
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     6.71  0.328   3.2   1.9
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan      2.7 0.0873   1.5 0.738

PDG      FULL      LWA    Deng    E1av
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN   -0.0117      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3       -76    -94.5   -75.8   -83.2
                $\chi_{c1}(1P)$ -24.4     -41.5    -52.6   -49.7   -42.4
                $\chi_{c2}(1P)$   NaN      1.83     2.56    2.46    1.94
                $\eta_{c}(2S)$    NaN -0.000608      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN      -119     -117    -134    -147
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN      -129     -122   -55.5    -101
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN      3.88     3.24    1.28    3.17
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN  -0.00612   -0.427 -0.0132  -0.106
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN    -0.311  -0.0152  -0.148  -0.088
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN   -0.0698 -0.00225 -0.0387 -0.0191

In [67]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output